### Win-rate draft (Original by Uncle Victor)

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

# backtest period
init_date, last_date = pd.Timestamp('2016-01-01'), pd.Timestamp('2020-12-31')

# download historical data for this period
tickers = ['AAPL', 'AMZN', 'GOOG', 'MSFT', 'NVDA', 'TSLA']
prices = yf.download(tickers, init_date, last_date)['Adj Close']
# display(prices)

'''step 1: collect as much trade info as possible (can drop later)'''

# generate a set of hypothetical simulation parameters
data = [f'{ticker}_A{p1:02d}_B{p2:02}' for ticker in tickers for p1 in range(1, 8) for p2 in range(1, 15)]
data = pd.DataFrame(data, columns=['config'])
# display(data)

# fill in dates and prices with historical data
# dates = pd.bdate_range(init_date, last_date-pd.Timedelta(days=30))
dates = prices.index[:-30]
size = data.shape[0] // len(tickers) # integer division
data['entry_date'] = pd.concat([dates.to_frame().reset_index(drop=True).sample(size, replace=False)] * len(tickers), axis=0).iloc[:, 0].values
data['entry_price'] = data[['config', 'entry_date']].apply(lambda x: prices.at[x[1], x[0].split('_')[0]], axis=1) # prices.at[ticker, date]
data['exit_date'] = data['entry_date'].apply(lambda x: prices.index[prices.index.get_loc(x)+np.random.choice(range(5, 25))]) # exit 5 to 24 days later
data['exit_price'] = data[['config', 'exit_date']].apply(lambda x: prices.at[x[1], x[0].split('_')[0]], axis=1) # prices.at[ticker, date]
data['long_pnl'] = data['exit_price'] - data['entry_price']
data['exit_year'] = data['exit_date'].dt.year
data[['ticker', 'param1', 'param2']] = data['config'].str.split('_').tolist()
display(data) # we have a set of random trades


'''step 2: collect the required stats'''

stats = []
for key, df in data.groupby(['ticker', 'exit_year']):
    ticker, year = key
    # display(df)
    
    trades = df.shape[0]
    winners = df[df['long_pnl'] > 0].shape[0]
    losers = df[df['long_pnl'] <= 0].shape[0]
    win_rate = winners / trades
    # print(f'{year}: {ticker}, {trades}, {winners}, {losers}, {win_rate*100.0:.1f}%')
    
    stats.append({'year': year, 'ticker': ticker, 'trades': trades, 'winners': winners, 'losers': losers, 'win_rate': win_rate})
stats = pd.DataFrame(stats)
# display(stats)

'''step 3: query the stats'''

stats = stats.sort_values(['year', 'win_rate'], ascending=[True, False]).reset_index(drop=True)
# stats = stats.sort_values(['year', 'winners'], ascending=[True, False]).reset_index(drop=True)
stats.index += 1
display(stats)

[*********************100%***********************]  6 of 6 completed


,config,entry_date,entry_price,exit_date,exit_price,long_pnl,exit_year,ticker,param1,param2
0,AAPL_A01_B01,2018-04-16,42.157066,2018-04-23,39.620258,-2.536808,2018,AAPL,A01,B01
1,AAPL_A01_B02,2020-05-18,77.882942,2020-05-26,78.320641,0.437698,2020,AAPL,A01,B02
2,AAPL_A01_B03,2020-03-03,71.349579,2020-04-06,64.728058,-6.621521,2020,AAPL,A01,B03
3,AAPL_A01_B04,2019-12-04,64.395210,2020-01-03,73.175919,8.780708,2020,AAPL,A01,B04
4,AAPL_A01_B05,2016-06-16,22.686298,2016-07-01,22.300245,-0.386053,2016,AAPL,A01,B05
...,...,...,...,...,...,...,...,...,...,...
583,TSLA_A07_B10,2016-09-07,40.341999,2016-09-16,41.080002,0.738003,2016,TSLA,A07,B10
584,TSLA_A07_B11,2020-10-16,439.670013,2020-11-11,417.130005,-22.540009,2020,TSLA,A07,B11
585,TSLA_A07_B12,2019-06-13,42.782001,2019-07-10,47.784000,5.001999,2019,TSLA,A07,B12
586,TSLA_A07_B13,2016-10-27,40.801998,2016-11-25,39.330002,-1.471996,2016,TSLA,A07,B13


RuntimeError: No active exception to reraise

### Tweaked Draft

In [ ]:
# backtest period
init_date, last_date = pd.Timestamp('2008-01-01'), pd.Timestamp('2021-12-31')

# download historical data for this period
my_asset_list = ['AAPL', 'MSFT', 'GOOGL', 'BRK-B', 'AMZN', 'TXN', 'NVDA', 'HON', 'NTES', 'ASML', 'AMAT', 'CSCO', 'PEP', 'COST', 'ADBE', 'CMCSA', 'INTC', 'AZN', 'QCOM', 'NFLX']
prices = yf.download(my_asset_list, init_date, last_date)['Adj Close']
# display(prices)

'''step 1: collect as much trade info as possible (can drop later)'''

# generate a set of hypothetical simulation parameters
data = [f'{asset}_A{p1:02d}_B{p2:02}' for asset in my_asset_list for p1 in range(1, 8) for p2 in range(1, 15)]
data = pd.DataFrame(data, columns = ['Parameter Configurations'])
# display(data)

# fill in dates and prices with historical data
# dates = pd.bdate_range(init_date, last_date-pd.Timedelta(days=30))
dates = prices.index[:-30]
size = data.shape[0] // len(my_asset_list) # integer division
data['Entry Date'] = pd.concat([dates.to_frame().reset_index(drop=True).sample(size, replace=False)] * len(my_asset_list), axis=0).iloc[:, 0].values
data['Entry Price'] = data[['Parameter Configurations', 'Entry Date']].apply(lambda x: prices.at[x[1], x[0].split('_')[0]], axis=1) # prices.at[ticker, date]
data['Exit Date'] = data['Entry Date'].apply(lambda x: prices.index[prices.index.get_loc(x)+np.random.choice(range(5, 25))]) # exit 5 to 24 days later
data['Exit Price'] = data[['Parameter Configurations', 'Exit Date']].apply(lambda x: prices.at[x[1], x[0].split('_')[0]], axis=1) # prices.at[ticker, date]
data['PnL'] = data['Exit Price'] - data['Entry Price']
data['Exit Year'] = data['Exit Date'].dt.year
data[['Asset', 'Param 1', 'Param 2']] = data['Parameter Configurations'].str.split('_').tolist()
# display(data) # we have a set of random trades


'''step 2: collect the required stats'''

stats = []
for key, df in data.groupby(['Asset', 'Exit Year']):
    asset, year = key
    # display(df)

    trades = df.shape[0]                 # total number of trades made
    winners = df[df['PnL'] > 0].shape[0] # number of winning trades
    losers = df[df['PnL'] <= 0].shape[0] # number of losing trades
    win_rate = winners / trades
    # print(f'{year}: {ticker}, {trades}, {winners}, {losers}, {win_rate*100.0:.1f}%')
    
    stats.append({'Year': year, 'Asset': asset, ' Total Trades': trades, 'Winning Trades': winners, 'Losing Trades': losers, 'Win Rate': win_rate})
stats = pd.DataFrame(stats)
# display(stats)


'''step 3: query the stats'''

stats = stats.sort_values(by = ['Win Rate'], ascending=False).reset_index(drop=True)
# stats = stats.sort_values(['year', 'winners'], ascending=[True, False]).reset_index(drop=True)
stats.index += 1
display(stats)
stats.to_excel(f'/Users/bervynwong/Desktop/CP Global Internship files/Dataframes/stats.xlsx', sheet_name = 'stats', index = False)